In [1]:
%pip install gdown
import gdown

url = "https://drive.google.com/drive/folders/1K1w5SBylgJXdjT_kqDBOSF6dgE5AlXit?usp=drive_link"
gdown.download_folder(url, quiet=True)

['/content/biometric_systems_project/dataset_base.zip',
 '/content/biometric_systems_project/haarcascade_eye.xml',
 '/content/biometric_systems_project/haarcascade_frontalface_default.xml',
 '/content/biometric_systems_project/haarcascade_mcs_mouth.xml',
 '/content/biometric_systems_project/haarcascade_mcs_nose.xml']

In [8]:
import zipfile
import os
import cv2
import shutil
import numpy as np
from google.colab.patches import cv2_imshow
from tqdm import tqdm
from google.colab import files

def download_folder_as_zip(folder_path, zip_name):
    """
    Comprime una cartella in un file ZIP e lo scarica.

    :param folder_path: Percorso della cartella da comprimere
    :param zip_name: Nome del file ZIP che verrà creato
    """
    if not os.path.exists(folder_path):
        print(f"La cartella {folder_path} non esiste.")
        return

    # Comprimere la cartella in un file ZIP
    shutil.make_archive(zip_name, 'zip', folder_path)

    # Scaricare il file ZIP
    zip_path = f"{zip_name}.zip"
    files.download(zip_path)
    print(f"Cartella compressa e scaricata come {zip_path}")

def show_images_in_folder(folder_path):
    """
    Mostra tutte le immagini in una determinata cartella.

    :param folder_path: Percorso della cartella contenente le immagini
    """
    if not os.path.exists(folder_path):
        print(f"La cartella {folder_path} non esiste.")
        return

    image_files = [file for file in os.listdir(folder_path) if file.lower().endswith(('.jpg', '.png', '.jpeg'))]
    if not image_files:
        print(f"Nessuna immagine trovata nella cartella {folder_path}.")
        return
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)

        if image is not None:
            cv2_imshow(image)  # Mostra l'immagine
        else:
            print(f"Impossibile caricare l'immagine: {image_file}")

def main():
    def update_dictionary(key, dictionary, file_name):
        if key not in dictionary:
            dictionary[key] = {file_name}
        else:
            dictionary[key].add(file_name)

    folder_path = "/content/biometric_systems_project"
    output_eyes = "/content/biometric_systems_output/output_eyes"
    output_mouth = "/content/biometric_systems_output/output_mouth"
    output_nose = "/content/biometric_systems_output/output_nose"
    os.makedirs(output_eyes, exist_ok=True)
    os.makedirs(output_mouth, exist_ok=True)
    os.makedirs(output_nose, exist_ok=True)

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".zip"):
            file_path = os.path.join(folder_path, file_name)
            extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
            os.makedirs(extract_folder, exist_ok=True)
            with zipfile.ZipFile(file_path, mode="r") as zip_ref:
                zip_ref.extractall(extract_folder)
                for root, _, files in os.walk(extract_folder):
                    for image_file in files:
                        if image_file.lower().endswith('.jpg'):
                            source_file = os.path.join(root, image_file)
                            destination_file = os.path.join(folder_path, image_file)
                            shutil.move(source_file, destination_file)
            shutil.rmtree(extract_folder)
            os.remove(extract_folder + ".zip")

    face_cascade_path = "/content/biometric_systems_project/haarcascade_frontalface_default.xml"
    face_cascade = cv2.CascadeClassifier(face_cascade_path)

    if face_cascade.empty():
        print("Errore: il file haarcascade_frontalface_default.xml non è stato trovato o caricato correttamente.")
        return

    total_images = len([name for name in os.listdir(folder_path) if name.lower().endswith('.jpg')])

    for file_name in tqdm(os.listdir(folder_path), desc="Elaborating images", total=total_images):
      image_path = os.path.join(folder_path, file_name)
      image = cv2.imread(image_path)
      if image is None:
          print(f"Immagine non valida: {file_name}.")
          continue

      gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

      if len(faces) == 1:  # Considera solo immagini con un volto rilevato
        for (x, y, w, h) in faces:
            # Calcola le sezioni del volto con proporzioni modificate
            eyes_lower = y + int(h * 0.5)  # Nuovo limite inferiore degli occhi (50%)
            nose_lower = y + int(h * 0.65)  # Nuovo limite inferiore del naso (65%)
            mouth_upper = y + int(h * 0.75)  # Limite superiore della bocca (75%)

            # Ritaglia le tre sezioni: occhi, naso, bocca
            eyes_section = image[y+int(h*0.25):eyes_lower, x:x+w]  # Nuova definizione degli occhi
            nose_section = image[eyes_lower:nose_lower, x+int(w*0.25):x+int(w*0.75)]  # Nuovo naso
            mouth_section = image[mouth_upper:y+h, x:x+w]

            # Salva le immagini delle sezioni
            base_name = os.path.splitext(file_name)[0]  # Nome base del file senza estensione
            eyes_path = os.path.join(output_eyes, f"{base_name}_eyes.jpg")
            nose_path = os.path.join(output_nose, f"{base_name}_nose.jpg")
            mouth_path = os.path.join(output_mouth, f"{base_name}_mouth.jpg")

            cv2.imwrite(eyes_path, eyes_section)
            cv2.imwrite(nose_path, nose_section)
            cv2.imwrite(mouth_path, mouth_section)
    else:
        print(f"Nessun volto o più volti rilevati in {file_name}.")

if __name__ == "__main__":
    main()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
download_folder_as_zip('/content/biometric_systems_output', 'datasets_eyes_nose_mouth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cartella compressa e scaricata come datasets_eyes_nose_mouth.zip
